#Проба пера

In [38]:
!pip install datasets

In [52]:
from datasets import Dataset, DatasetDict, load_dataset
import numpy as np
import pandas as pd
import json

In [ ]:
def create_json_dataset(split, dataset_size):
    dataset_json = {}
    dataset_json['rows'] = []
    for i in range(dataset_size):
      row = {}
      row['data'] = np.random.standard_normal(size=(1, 10)).tolist() 
      row['dictionary'] = {
          'number' : i,
          'some info' : np.random.normal(loc=0.0, scale=1.0),
          'split' : split
      }
      row['label'] = "label_" + str(i % 2)
      dataset_json['rows'].append(row)
      
    with open(split+'.json', 'w') as outfile:
        json.dump(dataset_json, outfile, indent=4)

In [ ]:
dataset_size = 100
create_json_dataset('train', dataset_size)
create_json_dataset('test', dataset_size)
create_json_dataset('val', dataset_size)

In [ ]:
dataset = load_dataset('json', data_files={'train': 'train.json', 'test': 'test.json', 'validation': 'val.json'}, 
                       field='rows')
dataset['validation']

Using custom data configuration default-8b14f3973da25a35


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8b14f3973da25a35/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['data', 'dictionary', 'label'],
    num_rows: 100
})

In [ ]:
df = dataset['train'].to_pandas()
df.head()

,data,dictionary,label
0,"[[0.06013459765918705, -0.9705806088760017, 2....","{'number': 0, 'some info': 1.2008960586656796,...",label_0
1,"[[1.2505976311535933, -0.9745440374678049, -0....","{'number': 1, 'some info': -0.206659760435971,...",label_1
2,"[[2.962235997532625, -0.2525350349440627, 0.01...","{'number': 2, 'some info': 0.7703592589587477,...",label_0
3,"[[0.1280320515177243, -0.6489202732059545, 1.0...","{'number': 3, 'some info': -0.863975549073052,...",label_1
4,"[[-0.12390010319112538, 0.28653869008702165, -...","{'number': 4, 'some info': 1.437995628326236, ...",label_0


In [ ]:
def add_sth_to_label(row):
    row['label'] = row['label'] + str(np.random.randint(0, 5))
    return row

dataset = dataset.map(add_sth_to_label)

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [ ]:
df = dataset['train'].to_pandas()
df.head()

,data,dictionary,label
0,"[[0.06013459765918705, -0.9705806088760017, 2....","{'number': 0, 'some info': 1.2008960586656796,...",label_03
1,"[[1.2505976311535933, -0.9745440374678049, -0....","{'number': 1, 'some info': -0.206659760435971,...",label_12
2,"[[2.962235997532625, -0.2525350349440627, 0.01...","{'number': 2, 'some info': 0.7703592589587477,...",label_04
3,"[[0.1280320515177243, -0.6489202732059545, 1.0...","{'number': 3, 'some info': -0.863975549073052,...",label_14
4,"[[-0.12390010319112538, 0.28653869008702165, -...","{'number': 4, 'some info': 1.437995628326236, ...",label_01


#Multiwoz with sentiment to pyarrow-dataset

In [2]:
import pickle

with open('/content/drive/MyDrive/ML&Stats labs/innoprac_deeppavlov/kmeans_convert_c2v_200_by_speaker_60_merged.pkl', 'rb') as f:
    cluster_data = pickle.load(f)

In [3]:
cluster_data.keys()

dict_keys(['dataset', 'cluster_embeddings', 'second_stage_clustering_labels'])

In [4]:
cluster_data['dataset'][0]

{'dialogue_id': 'PMUL4398.json',
 'turns': [{'first_stage_clustering_label': 79,
   'second_stage_clustering_label': 0,
   'speaker': 'USER',
   'turn_id': '0',
   'utterance': 'i need a place to dine in the center thats expensive'},
  {'first_stage_clustering_label': 309,
   'second_stage_clustering_label': 20,
   'speaker': 'SYSTEM',
   'turn_id': '1',
   'utterance': 'I have several options for you; do you prefer African, Asian, or British food?'},
  {'first_stage_clustering_label': 50,
   'second_stage_clustering_label': 11,
   'speaker': 'USER',
   'turn_id': '2',
   'utterance': 'Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation?'},
  {'first_stage_clustering_label': 283,
   'second_stage_clustering_label': 87,
   'speaker': 'SYSTEM',
   'turn_id': '3',
   'utterance': 'There is an Afrian place named Bedouin in the centre. How does that sound?'},
  {'first_stage_clustering_label': 14,
   'second_stage_clustering

In [5]:
cluster_data['cluster_embeddings'].shape

(400, 8)

In [6]:
import json

with open('/content/drive/MyDrive/ML&Stats labs/innoprac_deeppavlov/sentiment_emotion_train.txt', 'rb') as f:
    sentiment_data = json.load(f)
sentiment_data = sentiment_data['dialogues']

In [12]:
cluster_sentiment_dataset = {}
cluster_sentiment_dataset['clusters'] = []
for i in range(cluster_data['cluster_embeddings'].shape[0]):
    cluster_sentiment_dataset['clusters'].append({
        'cluster_id' : i,
        'cluster_embedding' : cluster_data['cluster_embeddings'][i],
        'utterances' : [],
        'sentiment' : {
            'POS' : 0,
            'NEG' : 0,
            'NEU' : 0
        },
        'size' : 0
    })

In [13]:
def comparator(dlg):
    return dlg['dialogue_id']
cluster_data['dataset'].sort(key=comparator)
sentiment_data.sort(key=comparator)

In [14]:
for i in range(len(sentiment_data)):
    for turn in sentiment_data[i]['turns']:
        uttr = {}
        uttr['utterance'] = turn['utterance']
        uttr['sentiment'] = turn['sentiment']
        uttr['speaker'] = turn['speaker']
        uttr['turn_id'] = turn['turn_id']
        uttr['dialogue_id'] = sentiment_data[i]['dialogue_id']
        uttr['label'] = max(turn['sentiment'], key=turn['sentiment'].get)

        cl_id = cluster_data['dataset'][i]['turns'][int(turn['turn_id'])]['first_stage_clustering_label']
        cluster_sentiment_dataset['clusters'][cl_id]['utterances'].append(uttr)
        cluster_sentiment_dataset['clusters'][cl_id]['size'] += 1
        cluster_sentiment_dataset['clusters'][cl_id]['sentiment'][uttr['label']] += 1

for cl in cluster_sentiment_dataset['clusters']:
    cl['sentiment']['POS'] /= cl['size']
    cl['sentiment']['NEG'] /= cl['size']
    cl['sentiment']['NEU'] /= cl['size']
    cl['label'] = max(cl['sentiment'], key=cl['sentiment'].get)

In [15]:
# change embedding np.array to list: .tolist()
with open('cluster_sentiment_label.txt', 'w') as outfile:
    json.dump(cluster_sentiment_dataset, outfile, indent=4)

In [ ]:
dataset = Dataset.from_dict(cluster_sentiment_dataset)
dataset.save_to_disk("sentiment_dataset")

# Adding intent to dataset


In [39]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse

from tqdm.notebook import tqdm
import typing as tp
import numpy.typing as ntp

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [40]:
import json
from pathlib import Path

def load_multiwoz(split, path='/content/drive/MyDrive/ML&Stats labs/innoprac_deeppavlov/multiwoz/data/MultiWOZ_2.2'):
    data_dir = Path(path) / split
    data = []
    data_parts = list(data_dir.iterdir())
    for data_part in tqdm(data_parts):
        with data_part.open() as f:
            data.extend(json.load(f))
    return data

In [41]:
from itertools import accumulate

class Utterance:
    def __init__(self, utterance: str, speaker: str, turn_id: str, **meta: tp.Any):
        self.utterance = utterance
        self.speaker = speaker
        self.turn_id = turn_id
        self.meta = meta
    
    def __str__(self) -> str:
        return self.utterance
    
    def __repr__(self) -> str:
        return f"[{self.turn_id:>2}] {self.speaker:>8}: \"{self.utterance}\""
    
    @classmethod
    def from_multiwoz_v22(cls, utterance: tp.Dict[str, tp.Any]) -> 'Utterance':
        return cls(**utterance)
    

class Dialogue:
    def __init__(self, utterances: tp.List[Utterance], dialogue_id: str, **meta: tp.Any):
        self.utterances = utterances
        self.dialogue_id = dialogue_id
        self.meta = meta
    
    def __len__(self) -> int:
        return len(self.utterances)
    
    def __str__(self) -> str:
        return "\n".join(str(utt) for utt in self.utterances)
    
    def __repr__(self) -> str:
        return f"[{self.dialogue_id}]\n" + '\n'.join(repr(utt) for utt in self.utterances)
    
    def __getitem__(self, i) -> Utterance:
        return self.utterances[i]
    
    def __iter__(self) -> tp.Iterator[Utterance]:
        return iter(self.utterances)
        
    
    @classmethod
    def from_multiwoz_v22(cls, dialogue: tp.Dict[str, tp.Any]) -> 'Dialogue':
        utterances = [Utterance.from_multiwoz_v22(utt) for utt in dialogue['turns']]
        dialogue_id = dialogue['dialogue_id']
        meta = {key: val for key, val in dialogue.items() if key not in ['turns', 'dialogue_id']}
        return cls(utterances=utterances, dialogue_id=dialogue_id, **meta)


class DialogueDataset(list):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.utterances = [utt.utterance for dialog in self for utt in dialog]
        
        self._dialogue_start = list(accumulate([0] + [len(dialogue) for dialogue in self]))
        
        self._utt_dialogue_id = [0] * len(self.utterances)
        self._utt_id = [0] * len(self.utterances)
        for d_start in self._dialogue_start[1:-1]:
            self._utt_dialogue_id[d_start] = 1
        current_utt_id = 0
        for i in range(len(self._utt_id)):
            if self._utt_dialogue_id[i] == 1:
                current_utt_id = 0
            self._utt_id[i] = current_utt_id
            current_utt_id += 1
        self._utt_dialogue_id = list(accumulate(self._utt_dialogue_id))
        
        self._dial_id_mapping = {dialogue.dialogue_id: i 
                                for i, dialogue in enumerate(self)}
    
    def get_dialogue_by_idx(self, idx: int) -> Dialogue:
        udi = self._utt_dialogue_id[idx]
        return self[udi]
    
    def get_utterance_by_idx(self, idx: int) -> Utterance:
        udi = self._utt_dialogue_id[idx]
        ui = self._utt_id[idx]
        return self[udi][ui]
    
    def get_dialog_start_idx(self, dialogue: 'Dialog') -> int:
        dialogue_idx = self._dial_id_mapping[dialogue.dialogue_id]
        d_start = self._dialogue_start[dialogue_idx]
        return d_start
    
    @classmethod
    def from_miltiwoz_v22(cls, multiwoz_v22: tp.List[tp.Dict[str, tp.Any]]) -> 'DialogueDataset':
        dialogues = [Dialogue.from_multiwoz_v22(dialog) for dialog in multiwoz_v22]
        return cls(dialogues)

class Subset(DialogueDataset):
    def __init__(self, dialogues: DialogueDataset, subset: tp.Iterable):
        subset_dialogues = [dialogues[idx] for idx in subset]
        super().__init__(subset_dialogues)

In [42]:
multiwoz = DialogueDataset.from_miltiwoz_v22(load_multiwoz('train'))

  0%|          | 0/17 [00:00<?, ?it/s]

In [43]:
services_dict = {}
for d in multiwoz:
    services_dict[d.dialogue_id] = d.meta['services']

In [44]:
with open('cluster_sentiment_label.txt') as json_file:
    clusters = json.load(json_file)

In [48]:
for cluster in clusters["clusters"]:
    cluster["services"] = {}
    for uttr in cluster["utterances"]:
        for serv in services_dict[uttr["dialogue_id"]]:
            if serv in cluster["services"]:
                cluster["services"][serv] += 1
            else:
                cluster["services"][serv] = 1
    for serv in cluster["services"].keys():
        cluster["services"][serv] /= cluster["size"]

In [49]:
clusters['clusters'][0:2]

[{'cluster_embedding': [-0.21947190165519714,
   1.8991761207580566,
   -1.1714054346084595,
   -3.5083158016204834,
   -2.9204437732696533,
   1.0794681310653687,
   0.8966759443283081,
   1.7798773050308228],
  'cluster_id': 0,
  'label': 'NEU',
  'sentiment': {'NEG': 0.0,
   'NEU': 0.8324324324324325,
   'POS': 0.16756756756756758},
  'services': {'attraction': 0.1783783783783784,
   'hotel': 0.16216216216216217,
   'restaurant': 1.0,
   'taxi': 0.16756756756756758,
   'train': 0.17297297297297298},
  'size': 185,
  'utterances': [{'dialogue_id': 'MUL0083.json',
    'label': 'NEU',
    'sentiment': {'NEG': 0.0013128481805324554,
     'NEU': 0.8529528379440308,
     'POS': 0.1457342505455017},
    'speaker': 'USER',
    'turn_id': '0',
    'utterance': 'Hi, I am looking for a British-themed restaurant in the centre.'},
   {'dialogue_id': 'MUL0114.json',
    'label': 'NEU',
    'sentiment': {'NEG': 0.002007097005844116,
     'NEU': 0.8684070706367493,
     'POS': 0.12958583235740662},

In [50]:
with open('cluster_sentiment_service_label.txt', 'w') as outfile:
    json.dump(clusters, outfile, indent=4)

In [53]:
dataset = Dataset.from_dict(clusters)
dataset.save_to_disk("sentiment_service_dataset")